In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import pandas as pd
test = pd.read_csv("../input/codenation-enem2/test.csv")
train = pd.read_csv("../input/codenation-enem2/train.csv")
df_resposta = pd.DataFrame()

In [0]:
test

In [0]:
train

In [0]:
print(set(test.columns).issubset(set(train.columns)))

In [0]:
df_resposta['NU_INSCRICAO'] = test['NU_INSCRICAO']

In [0]:
test = test.select_dtypes(include=['int64','float64']) # selecionar todos os campos com o tipo escolhido (float)
# train = train.select_dtypes(include=['int64','float64'])

In [0]:
features = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']
plt.figure(figsize=(4,4))
sns.heatmap(test[features].corr(), annot=True, linewidth=0.5, linecolor='black', cmap='Greens')
plt.xticks(rotation=90)
plt.show()

In [0]:
train = train.loc[(train['NU_NOTA_CN'].notnull()) & (train['NU_NOTA_CH'].notnull())  & (train['NU_NOTA_LC'].notnull()) & (train['NU_NOTA_REDACAO'].notnull()) & (train['NU_NOTA_MT'].notnull())]
# test = test.loc[(test['NU_NOTA_CN'].notnull()) & (test['NU_NOTA_CH'].notnull())  & (test['NU_NOTA_LC'].notnull()) & (test['NU_NOTA_REDACAO'].notnull())]

In [0]:
# test[features].isnull().sum() # Não existe mais linhas com valores nulos
test[features].notnull().sum() # Não existe mais linhas com valores nulos

In [0]:
# train['NU_NOTA_CN'].fillna(train['NU_NOTA_CN'].mean(), inplace=True)
# train['NU_NOTA_CH'].fillna(train['NU_NOTA_CH'].mean(), inplace=True)
# train['NU_NOTA_REDACAO'].fillna(train['NU_NOTA_REDACAO'].mean(), inplace=True)
# train['NU_NOTA_LC'].fillna(train['NU_NOTA_LC'].mean(), inplace=True)
# test['NU_NOTA_CN'].fillna(train['NU_NOTA_CN'].mean(), inplace=True)
# test['NU_NOTA_CH'].fillna(train['NU_NOTA_CH'].mean(), inplace=True)
# test['NU_NOTA_REDACAO'].fillna(train['NU_NOTA_REDACAO'].mean(), inplace=True)
# test['NU_NOTA_LC'].fillna(train['NU_NOTA_LC'].mean(), inplace=True)

train.NU_NOTA_CN.fillna(train.NU_NOTA_CN.mean(), inplace=True)
train.NU_NOTA_CH.fillna(train.NU_NOTA_CH.mean(), inplace=True)
train.NU_NOTA_REDACAO.fillna(train.NU_NOTA_REDACAO.mean(), inplace=True)
train.NU_NOTA_LC.fillna(train.NU_NOTA_LC.mean(), inplace=True)
test.NU_NOTA_CN.fillna(train.NU_NOTA_CN.mean(), inplace=True)
test.NU_NOTA_CH.fillna(train.NU_NOTA_CH.mean(), inplace=True)
test.NU_NOTA_REDACAO.fillna(train.NU_NOTA_REDACAO.mean(), inplace=True)
test.NU_NOTA_LC.fillna(train.NU_NOTA_LC.mean(), inplace=True)

## O que queremos descobrir ?
# #     Predizer a nota de matematica

In [0]:
y = train['NU_NOTA_MT']
y

### Treinando com tabela de notas de matematica com 10097 linhas

In [0]:
# Definição do dataset de treino somente com as informações relevantes para treinar o modelo
# features = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO','NU_NOTA_MT']
features = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']
x_train = train[features]
x_train

In [0]:
scaler = preprocessing.StandardScaler().fit(x_train)
scaler

In [0]:
X_train_scaled = scaler.transform(x_train)
print('Média: {}'.format(X_train_scaled.mean(axis=1)))
print('Desvio padrao: {}'.format(X_train_scaled.std(axis=0)))

In [0]:
features = ['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_REDACAO']
x_test = test[features]

In [0]:
pipeline = make_pipeline(preprocessing.StandardScaler(), RandomForestRegressor(n_estimators=200, n_jobs=-1, warm_start=True))

In [0]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [0]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
clf.fit(X_train_scaled, y)

In [0]:
print(clf.best_params_)

In [0]:
x_test

In [0]:
pred_notas = clf.predict(x_test)

In [0]:
print(pred_notas)
print('Tamanho: {}'.format(pred_notas.size))

In [0]:
pred_notas

In [0]:
np.around(pred_notas,2)

In [0]:
df_resposta['NU_NOTA_MT'] = pd.DataFrame(data=np.around(pred_notas,2))
df_resposta

In [0]:
df_resposta.groupby('NU_NOTA_MT').size()

In [0]:
df_resposta['NU_NOTA_MT'].unique()

In [0]:
df_resposta.to_csv('answer.csv', index=False, header=True)

In [0]:

# Salvar o modelo preditivo
joblib.dump(clf, 'rf_regressor.pkl')

# Usar/carregar o modelo preditivo
clf2 = joblib.load('rf_regressor.pkl')
clf2.predict(x_test)

###### Código baseado no artigo https://medium.com/ensina-ai/machine-learning-randomforest-para-prever-nota-de-matem%C3%A1tica-do-enem-2016-8893b73882f4 (02 de dezem. de 2019)